![](https://t4.ftcdn.net/jpg/04/60/05/05/360_F_460050532_7JjxeTTaZLyk7RTOayql8iX4O6Zlctjs.jpg)

#### This year is the year of the black tiger. happy New Year.
#### (올해는 흑호의 해입니다. 새해 복 많이 받으세요.)

#### Create a model using the following library.: (다음 라이브러리를 사용하여 모델을 생성했다.:)
#### ● Pycaret (파이캐롯)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
!pip install pycaret[full]

# Loading Data (데이터 불러오기)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
import pandas as pd
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.regression import *
from sklearn.datasets import make_classification
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

data = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')

train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv',index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv',index_col='row_id')

#### ▶ This is the code to check the total file size.
####   If you put 'train' and 'test' in '( )', you can check the capacity of each.
####   (전체 파일 용량을 확인하는 코드이다. '( )' 안에 'train'과 'test'를 넣으면 각각의 용량도 확인할 수 있다.)

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
mem_usage(train)

In [ ]:
mem_usage(test)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
data.head().style.applymap(lambda x : "background-color: #bff9ff")\
.set_table_styles([{'selector' : '', 'props' : [('border', '2px solid black')]}])

In [ ]:
data = data.drop('row_id', axis=1)
data.head()

data.date = pd.to_datetime(data.date)
data.date.dtype

In [ ]:
prop = pd.DataFrame(data.groupby('product')['num_sold'].sum()).reset_index()

prop.num_sold = prop.num_sold/prop.num_sold.sum()
prop.style.applymap(lambda x : "background-color: #bff9ff")\
.set_table_styles([{'selector' : '', 'props' : [('border', '2px solid black')]}])

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
train.describe

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
for col in ['country', 'store', 'product']:
    print(col, train[col].unique())

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
print('train date duration : ', train['date'].min(), train['date'].max())
print('test date duration : ', test['date'].min(), test['date'].max())

#### Train dataset is data for 3 years from 2015 to 2018, and test dataset is data for 1 year from 2019.
#### (Train 데이터셋은 2015년부터 2018년까지 3년간의 데이터이고, 테스트 데이터셋은 2019년부터 1년간의 데이터입니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
for col in ['country', 'store', 'product']:
    display(pd.DataFrame(train[col].value_counts()))

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
for col in ['country', 'store', 'product']:
    display(pd.DataFrame(test[col].value_counts()))

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

# Preprocessing & Visualization (전처리와 시각화)
#### Time series data needs to be preprocessed for data analysis, feature engineering, visualization, etc.
#### (시계열 데이터는 데이터 분석, 피쳐 엔지니어링, 시각화 등을 위해 전처리해야 합니다.)

#### ▶ The date column type is text. Convert to datetime type for easy handling in pandas. 
#### (날짜 열 유형은 텍스트입니다. pandas에서 쉽게 처리할 수 있도록 datetime형으로 변환합니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

#### Pivot Table (time) (피벗 테이블(시간))
#### ▶ by date (날짜 별)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
train_date = train.set_index('date').pivot(columns=['country', 'store', 'product'], values='num_sold')
train_date.head()

#### ▶ by month (월 별)
#### ▶ For grouping by month, I recommend using pandas' latest feature grouper.
####   (월 별 그룹화는 pandas의 최신 기능 그룹화를 사용하는 것이 좋습니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
train_month = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country', 'store', 'product'])['num_sold'].mean().unstack([1, 2, 3])
train_month_country = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country'])['num_sold'].mean().unstack()
train_month.head()

In [ ]:
norway  = data[data.country == 'Norway']
norway  = pd.DataFrame(norway.groupby('product')['num_sold'].sum()).reset_index()

finland = data[data.country == 'Finland']
finland = pd.DataFrame(finland.groupby('product')['num_sold'].sum()).reset_index()

sweden  = data[data.country == 'Sweden']
sweden  = pd.DataFrame(sweden.groupby('product')['num_sold'].sum()).reset_index()

In [ ]:
fig = make_subplots(rows=3, cols=1,
                    specs=[[{'type':'domain'}],[{'type':'domain'}],[{'type':'domain'}]],
                    vertical_spacing = 0.1)

fig.add_trace(go.Pie(
             values = norway['num_sold'],
             labels = norway['product'],
             title = dict(text = 'Distribution of<br>Kaggle Products<br>sales in Norway',
                          font = dict(size=18, family = 'monospace'),
                          ),
             hole = 0.5,
             hoverinfo='label+percent',),1,1)

fig.add_trace(go.Pie(
             values = finland['num_sold'],
             labels = finland['product'],
             title = dict(text = 'Distribution of<br>Kaggle Products<br>sales in Finland',
                          font = dict(size=18, family = 'monospace'),
                          ),
             hole = 0.5,
             hoverinfo='label+percent',),2,1)

fig.add_trace(go.Pie(
             values = sweden['num_sold'],
             labels = sweden['product'],
             title = dict(text = 'Distribution of<br>Kaggle Products<br>sales in Sweden',
                          font = dict(size=18, family = 'monospace'),
                          ),
             hole = 0.5,
             hoverinfo='label+percent',),3,1)

fig.update_traces(row=1, col=1, hoverinfo='label+percent',
                  textinfo='label+percent',
                  textfont_size=12,
                  opacity = 0.8,
                  showlegend = False,
                  marker = dict(colors = sns.color_palette('Reds').as_hex(),
                              line=dict(color='#000000', width=1)))

fig.update_traces(row=2, col=1, hoverinfo='label+percent',
                  textinfo='label+percent',
                  textfont_size=12,
                  opacity = 0.8,
                  showlegend = False,
                  marker = dict(colors = sns.color_palette('Blues').as_hex(),
                              line=dict(color='#000000', width=1)))

fig.update_traces(row=3, col=1, hoverinfo='label+percent',
                  textinfo='label+percent',
                  textfont_size=12,
                  opacity = 0.8,
                  showlegend = False,
                  marker = dict(colors = sns.color_palette('Wistia').as_hex(),
                              line=dict(color='#000000', width=1)))

fig.add_layout_image(
    dict(
        source="https://i.postimg.cc/mZ8B9tGT/image.png",
        xref="paper", yref="paper",
        x=0.05, y=0.92, opacity = 0.8,
        sizex=0.2, sizey=0.2,
        xanchor="left", yanchor="top"
    )
)

fig.add_layout_image(
    dict(
        source="https://i.postimg.cc/wv9Yydft/image.png",
        xref="paper", yref="paper",
        x=0.05, y=0.55, opacity = 0.8,
        sizex=0.2, sizey=0.2,
        xanchor="left", yanchor="top"
    )
)

fig.add_layout_image(
    dict(
        source="https://i.postimg.cc/T27vwFrw/image.png",
        xref="paper", yref="paper",
        x=0.05, y=0.18, opacity = 0.8,
        sizex=0.2, sizey=0.2,
        xanchor="left", yanchor="top"
    )
)

fig.update_layout(margin=dict(t=0, b=0, l=0, r=0),
                  paper_bgcolor = '#d5eff0',
                  height = 1200,
                  font_family   = 'monospace',)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x='country', data=train, order=train["country"].value_counts().index[:3],palette = "turbo")
plt.title("Total counts of country ", size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x='store', data=train, order=train["store"].value_counts().index[:2],palette = "turbo")
plt.title("Total counts of stores ", size=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,2))
sns.countplot(x='product', data=train, order=train["product"].value_counts().index[:3],palette = "flag")
plt.title("Total count kaggle products ", size=13)
plt.show()

In [ ]:
train['date'] = pd.to_datetime(train['date'], format='%Y/%m/%d')
plt.figure(figsize=(15,7))
sld_time = train.groupby(['date']).sum().reset_index()
sns.lineplot(x=sld_time.date, y=sld_time.num_sold,)
plt.title('number sold over time ', fontsize=14)
plt.show()

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
country_daily = train.groupby(['date','country'])[['num_sold']].sum().reset_index(level=[0,1])
for country in train['country'].unique():
    display(country_daily[country_daily['country']==country].sort_values(by=['num_sold'], ascending=False).head(30))

#### Grouping by month makes it easier to see overall trends. You can see the trend of peaking at the beginning and end of the year and decreasing before and after.
#### (월별로 그룹화하면 전반적인 추세를 더 쉽게 볼 수 있습니다. 
####  연초와 연말에 정점을 찍고 전후로 감소하는 추세를 볼 수 있습니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_monthly_country = train.set_index('date').groupby([pd.Grouper(freq='M'),'country'])[['num_sold']].mean()
sns.lineplot(x="date", y='num_sold', hue='country', data=train_monthly_country, ax=ax)

ax.set_ylabel('num_sold')
ax.set_title('Monthly Trend by Country', fontsize=15, fontweight='bold', loc='left')
ax.grid(alpha=0.5)
plt.show()

#### Day of Week (요일)
#### Time-series data, such as product sales, often have different distributions on weekends and weekdays. Using the day of the week as a feature is often very effective.
#### (제품 판매와 같은 시계열 데이터는 종종 주말과 주중의 분포가 다릅니다. 요일을 기능으로 사용하면 종종 매우 효과적입니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
train['dayofweek'] = train['date'].dt.dayofweek
test['dayofweek'] = test['date'].dt.dayofweek

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_dayofweek = train.set_index('date').groupby([pd.Grouper(freq='M'), 'dayofweek'])[['num_sold']].mean()

sns.lineplot(x="date", y='num_sold', hue='dayofweek', data=train_dayofweek, ax=ax)
ax.set_title('Trend by day of the week', fontsize=15, fontweight='bold', loc='left')
ax.grid(alpha=0.5)
plt.show()

#### The following is a comparison by creating a weekend column in more detail.
#### (다음은 주말 칼럼을 보다 구체적으로 생성하여 비교한 것이다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
train['weekend'] = train['dayofweek'].apply(lambda x : x >= 5)
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_weekend = train.set_index('date').groupby([pd.Grouper(freq='M'), 'weekend'])[['num_sold']].mean()
sns.lineplot(x="date", y="num_sold", hue='weekend', data=train_weekend, ax=ax)
ax.set_title('Weekend vs. Weekday Trend Comparison', fontsize=15, fontweight='bold', loc='left')
ax.grid(alpha=0.5)
plt.show()

In [ ]:
country_dayofweek = pd.pivot_table(train, index='country', columns='dayofweek', values='num_sold', aggfunc=np.mean)
country_dayofweek = pd.DataFrame(country_dayofweek.divide(country_dayofweek.sum(axis=1), axis=0).unstack()).reset_index(level=[0,1])
country_dayofweek.rename(columns={0:'num_sold'}, inplace=True)
country_dayofweek

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
fig, ax = plt.subplots(figsize=(12, 9))
country_dayofweek = pd.pivot_table(train, index='country', columns='dayofweek', values='num_sold', aggfunc=np.mean)
country_dayofweek = pd.DataFrame(country_dayofweek.divide(country_dayofweek.sum(axis=1), axis=0).unstack()).reset_index(level=[0,1])
country_dayofweek.rename(columns={0:'num_sold'}, inplace=True)
# country_dayofweek.reset_index(level=[0,1])
sns.barplot(x='dayofweek', y='num_sold', hue='country',data=country_dayofweek, ax=ax)
ax.grid(axis='y',alpha=0.5, )
ax.set_xticklabels(['MON', 'TUE', 'WED','THU','FRI','SAT','SUN'])
ax.set_title('Percentage by day of the week by country', fontsize=15, fontweight='bold', loc='left')
plt.show()

#### You can see that Friday has a higher percentage compared to other days of the week.
#### (금요일에는 다른 요일에 비해 비율이 높은 것을 알 수 있습니다.)

#### Pivot (etc) (피벗(기타))
####  ▶ product ratio by country  (국가별 제품 비율)
#### You can check the following to see if there is a preference for each country, and there does not seem to be a significant difference.
#### (국가별 선호도가 있는지는 다음과 같이 확인할 수 있으며, 큰 차이는 없어 보입니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
country_product = pd.pivot_table(train, index='country', columns='product', values='num_sold', aggfunc=np.mean)
country_product.divide(country_product.sum(axis=1), axis=0)

#### You can check the following to see if there is a preference for each day of week, and there does not seem to be a significant difference.
#### (요일별 선호도가 있는지 다음과 같이 확인할 수 있으며, 큰 차이는 없어 보입니다.)

In [ ]:
# Code Source : https://www.kaggle.com/subinium/tps-jan-happy-new-year
country_product_dayofweek = pd.pivot_table(train, index='dayofweek', columns='product', values='num_sold', aggfunc=np.mean)
country_product_dayofweek.divide(country_product_dayofweek.sum(axis=1), axis=0)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
def pre_process(df):
    
    df['date'] = pd.to_datetime(df['date'])
    df['week']= df['date'].dt.week
    df['year'] = 'Y'+df['date'].dt.year.astype(str)
    df['quarter'] = 'Q'+df['date'].dt.quarter.astype(str)
    df['day'] = df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df.loc[(df.date.dt.is_leap_year) & (df.dayofyear >= 60),'dayofyear'] -= 1
    df['weekend'] = df['date'].dt.weekday >=5
    df['weekday'] = 'WD' + df['date'].dt.weekday.astype(str)
    df.drop(columns=['date'],inplace=True)   

pre_process(train)
pre_process(test)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
train.info(), test.info()

# Modeling (모델링)

#### Code to calculate the 'time series data' of the second place in the 'Web Traffic Time Series Prediction' contest held in Kaggle 4 years ago
#### (4년전 캐글에서 열린 '웹 트래픽 시계열 예측' 대회에서 2등을 한 사람의 '시계열 데이터'를 계산하는 코드)

In [ ]:
# Code sorce : https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [ ]:
reg = setup(data = train,
            target = 'num_sold',
            normalize=True,
            normalize_method='robust',
            transform_target = True,
            data_split_shuffle = False, 
            create_clusters = False,
            use_gpu = True,
            silent = True,
            fold=10,
            n_jobs = -1)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
add_metric('SMAPE', 'SMAPE', SMAPE, greater_is_better = False)
top =compare_models(sort = 'SMAPE',n_select = 3, include = ['catboost','lightgbm','xgboost']) 

In [ ]:
best_models = compare_models(top)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
blend = blend_models(top)
predict_model(blend)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
final_blend = finalize_model(blend)  
predict_model(final_blend)

In [ ]:
# Code Sorce : https://www.kaggle.com/bernhardklinger/tps-jan-2022
preds = predict_model(final_blend, data=test) 
sub = pd.DataFrame(list(zip(test.index,preds.Label)),columns = ['row_id', 'num_sold']) 
sub.to_csv('submission.csv', index = False) 
print(sub.head(),sub.describe())

In [ ]:
test.head()

In [ ]:
train.head()